In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

class VehicleHumanAnimalDetector:
    def __init__(self, model_size='medium', authorized_dir="authorized_persons"):
        size_map = {'nano': 'n', 'small': 's', 'medium': 'm', 'large': 'l', 'xlarge': 'x'}
        self.model = YOLO(f'yolov8{size_map[model_size]}.pt')
        
        # Target classes
        self.target_classes = {
            'car': 2, 'truck': 7, 'bus': 5, 'motorcycle': 3, 'bicycle': 1,  # Vehicles
            'person': 0,  # Humans
            'dog': 16, 'cat': 17, 'horse': 19, 'sheep': 18, 'cow': 20,      # Animals
            'bear': 21, 'bird': 15, 'elephant': 22, 'zebra': 23, 'giraffe': 24
        }
        
        # Colors
        self.colors = {
            'vehicle': (0, 255, 0),       # Green
            'human_authorized': (0, 255, 255),  # Yellow
            'human_unauthorized': (0, 0, 255),  # Red
            'animal': (255, 0, 0)         # Blue
        }
        
        # Face recognition setup
        self.face_recognizer = cv2.face.LBPHFaceRecognizer_create()
        self.authorized_faces = []
        self.authorized_names = []
        self._load_authorized_faces(authorized_dir)

    def _load_authorized_faces(self, directory):
        """Load and train faces from the authorized directory."""
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        
        faces = []
        labels = []
        label_id = 0
        
        for filename in os.listdir(directory):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                path = os.path.join(directory, filename)
                image = cv2.imread(path)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                # Detect faces in the authorized image
                detected_faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                for (x, y, w, h) in detected_faces:
                    faces.append(gray[y:y+h, x:x+w])
                    labels.append(label_id)
                    self.authorized_names.append(os.path.splitext(filename)[0])
                    label_id += 1
        
        if faces:
            self.face_recognizer.train(faces, np.array(labels))

    def _is_authorized(self, face_img):
        """Check if the detected face matches an authorized face."""
        gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        label, confidence = self.face_recognizer.predict(gray)
        
        # Lower confidence = better match (adjust threshold as needed)
        if confidence < 100:
            return True, self.authorized_names[label]
        return False, "Unauthorized"

    def detect(self, frame):
        """Process a frame and annotate detections."""
        results = self.model(frame)
        annotated_frame = frame.copy()
        
        for box in results[0].boxes:
            class_id = int(box.cls)
            class_name = self.model.names[class_id]
            
            if class_name not in self.target_classes:
                continue
            
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            if class_name == 'person':
                face_img = frame[y1:y2, x1:x2]
                is_authorized, name = self._is_authorized(face_img)
                color = self.colors['human_authorized'] if is_authorized else self.colors['human_unauthorized']
                label = name if is_authorized else "Unauthorized"
            elif class_name in ['car', 'truck', 'bus', 'motorcycle', 'bicycle']:
                color = self.colors['vehicle']
                label = class_name
            else:
                color = self.colors['animal']
                label = class_name
            
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated_frame, label, (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        return annotated_frame

    def run_realtime(self, camera_id=0):
        """Run real-time detection."""
        cap = cv2.VideoCapture(camera_id)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            annotated_frame = self.detect(frame)
            cv2.imshow("Detection", annotated_frame)
            
            if cv2.waitKey(1) == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    detector = VehicleHumanAnimalDetector(
        model_size='medium',
        authorized_dir="authorized" 
    )
    detector.run_realtime()


0: 480x640 1 person, 849.5ms
Speed: 2.0ms preprocess, 849.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)



QObject::moveToThread: Current thread (0x2068efc0) is not the object's thread (0x2c56b760).
Cannot move to target thread (0x2068efc0)

QObject::moveToThread: Current thread (0x2068efc0) is not the object's thread (0x2c56b760).
Cannot move to target thread (0x2068efc0)

QObject::moveToThread: Current thread (0x2068efc0) is not the object's thread (0x2c56b760).
Cannot move to target thread (0x2068efc0)

QObject::moveToThread: Current thread (0x2068efc0) is not the object's thread (0x2c56b760).
Cannot move to target thread (0x2068efc0)

QObject::moveToThread: Current thread (0x2068efc0) is not the object's thread (0x2c56b760).
Cannot move to target thread (0x2068efc0)

QObject::moveToThread: Current thread (0x2068efc0) is not the object's thread (0x2c56b760).
Cannot move to target thread (0x2068efc0)

QObject::moveToThread: Current thread (0x2068efc0) is not the object's thread (0x2c56b760).
Cannot move to target thread (0x2068efc0)

QObject::moveToThread: Current thread (0x2068efc0) is n

0: 480x640 1 person, 1025.2ms
Speed: 3.6ms preprocess, 1025.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 928.3ms
Speed: 4.4ms preprocess, 928.3ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 couch, 917.8ms
Speed: 3.1ms preprocess, 917.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 835.3ms
Speed: 3.1ms preprocess, 835.3ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 820.7ms
Speed: 4.0ms preprocess, 820.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 815.3ms
Speed: 2.6ms preprocess, 815.3ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 811.6ms
Speed: 3.0ms preprocess, 811.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 798.1ms
Speed: 3.7ms preprocess, 798.1ms inference, 1.2ms postprocess pe

KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

class VehicleHumanAnimalDetector:
    def __init__(self, model_size='medium', authorized_dir="authorized_persons"):
        # Define model size mapping
        size_map = {
            'nano': 'n',
            'small': 's',
            'medium': 'm',
            'large': 'l',
            'xlarge': 'x'
        }
        
        # Load YOLO model
        self.model = YOLO(f'yolov8{size_map[model_size]}.pt')
        
        # Target classes (vehicles, humans, animals)
        self.target_classes = {
            'car': 2, 'truck': 7, 'bus': 5, 'motorcycle': 3, 'bicycle': 1,  # Vehicles
            'person': 0,  # Humans
            'dog': 16, 'cat': 17, 'horse': 19, 'sheep': 18, 'cow': 20,      # Animals
            'bear': 21, 'bird': 15, 'elephant': 22, 'zebra': 23, 'giraffe': 24
        }
        
        # Colors for visualization
        self.colors = {
            'vehicle': (0, 255, 0),       # Green
            'human_authorized': (0, 255, 255),  # Yellow (authorized)
            'human_unauthorized': (0, 0, 255),  # Red (unauthorized)
            'animal': (255, 0, 0)         # Blue
        }
        
        # Face recognition setup
        self.face_recognizer = cv2.face.LBPHFaceRecognizer_create()
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.authorized_names = []
        self._load_authorized_faces(authorized_dir)

    def _load_authorized_faces(self, directory):
        """Load and train authorized faces."""
        faces = []
        labels = []
        label_id = 0

        print(f"Loading authorized faces from {directory}...")
        for filename in os.listdir(directory):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                path = os.path.join(directory, filename)
                image = cv2.imread(path)
                if image is None:
                    print(f"⚠️ Failed to load {filename}")
                    continue

                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                detected_faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)

                if len(detected_faces) == 0:
                    print(f"❌ No faces found in {filename}")
                else:
                    for (x, y, w, h) in detected_faces:
                        face_roi = gray[y:y+h, x:x+w]
                        faces.append(face_roi)
                        labels.append(label_id)
                        self.authorized_names.append(os.path.splitext(filename)[0])
                        print(f"✅ Loaded face from {filename} (Label: {label_id})")
                        label_id += 1

        if len(faces) > 0:
            self.face_recognizer.train(faces, np.array(labels))
            print(f"Trained on {len(faces)} authorized faces.")
        else:
            print("❌ No valid faces found for training!")

    def _is_authorized(self, face_img):
        """Check if the detected face matches an authorized face."""
        gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)  # Improve contrast
        
        # Detect face (required for LBPH)
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            print("⚠️ No face detected in the current frame!")
            return False, "Unauthorized"

        x, y, w, h = faces[0]
        face_roi = gray[y:y+h, x:x+w]
        
        label, confidence = self.face_recognizer.predict(face_roi)
        print(f"Prediction: Label={label}, Confidence={confidence:.1f}")
        
        if confidence < 70:  # Lower = better match (adjust as needed)
            return True, self.authorized_names[label]
        return False, "Unauthorized"

    def detect(self, frame):
        """Process a frame and annotate detections."""
        results = self.model(frame)
        annotated_frame = frame.copy()

        for box in results[0].boxes:
            class_id = int(box.cls)
            class_name = self.model.names[class_id]
            
            if class_name not in self.target_classes:
                continue
            
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            if class_name == 'person':
                face_img = frame[y1:y2, x1:x2]
                is_authorized, name = self._is_authorized(face_img)
                color = self.colors['human_authorized'] if is_authorized else self.colors['human_unauthorized']
                label = name if is_authorized else "Unauthorized"
                print(f"Person detected: {label}")
            else:
                category = 'vehicle' if class_name in ['car', 'truck', 'bus'] else 'animal'
                color = self.colors[category]
                label = class_name

            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated_frame, label, (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        return annotated_frame

    def run_realtime(self, camera_id=0):
        """Run real-time detection."""
        cap = cv2.VideoCapture(camera_id)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            annotated_frame = self.detect(frame)
            cv2.imshow("Detection", annotated_frame)
            
            if cv2.waitKey(1) == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    detector = VehicleHumanAnimalDetector(
        model_size='medium',
        authorized_dir="Project/authorized"  # Your directory
    )
    
    # Test on an image
    image = cv2.imread("Project/authorized/p1.jpg")
    if image is not None:
        result = detector.detect(image)
        cv2.imshow("Result", result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Error: Could not load the test image.")